In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[1],1
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 3, 6]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  6.011740                    1  3.454205
0.000750        2  4.585616                    2  3.251484
0.001052        3  4.307809                    3  3.296962
0.001476        4  3.987559                    4  3.280302
0.002070        5  3.639015                    5  3.182588
0.002904        6  3.279372                    6  2.993593
0.004074        7  2.921127                    7  2.713081
0.005714        8  2.572211                    8  2.368735
0.008015        9  2.237861                    9  2.009856
0.011243       10  1.923741                   10  1.694601
0.015771       11  1.635788                   11  1.461036
0.022122       12  1.380278                   12  1.297041
0.031031       13  1.165474                   13  1.162134
0.043528       14  0.973910                   14  1.004668
0.061057       15  0.795869                   15  0.814598
0.085645       16  0.657084                   16  0.642961
0.120136       17  0.563709                   17  0.528802
0.168516       18  0.505366                   18  0.477362
0.236378       19  0.476648                   19  0.466726
0.331549       20  0.476110                   20  0.471966
0.465100       21  0.511318                   21  0.496359
0.652400       22  0.518116                   22  0.496199
0.915100       23  0.478928                   23  0.475611
1.283650       24  0.444843                   24  0.470742
1.800600       25  0.418484                   25  0.457571
2.525700       26  0.391751                   26  0.419546
3.542800       27  0.361303                   27  0.377822
4.969550       28  0.335358                   28  0.329033
6.970850       29  0.321727                   29  0.301856
9.778100       30  0.310393                   30  0.296867
13.715850      31  0.295116                   31  0.287567
19.239350      32  0.275577                   32  0.265466
26.987250      33  0.264919                   33  0.260153
37.855300      34  0.265604                   34  0.263541
53.100050      35  0.264156                   35  0.259274
73.887500      36  0.261206                   36  0.253519
97.662500      37  0.257209                   37  0.241391
121.437500     38  0.250354                   38  0.229547
145.212500     39  0.319512                   39  0.295327
168.987500     40  0.371412                   40  0.353047
192.762500     41  0.410160                   41  0.396119
216.537500     42  0.410683                   42  0.396449
240.312500     43  0.400317                   43  0.384968
264.087500     44  0.385069                   44  0.375801
287.862500     45  0.354458                   45  0.356339
311.637500     46  0.342206                   46  0.352399
335.412500     47  0.377542                   47  0.392209
359.187500     48  0.421650                   48  0.432619
382.962500     49  0.547598                   49  0.550015
406.737500     50  0.674417                   50  0.669411
430.512500     51  0.723477                   51  0.734151
454.287500     52  0.710377                   52  0.750096
478.062500     53  0.658486                   53  0.696843
501.837500     54  0.584805                   54  0.594322
525.612500     55  0.499319                   55  0.497929
549.387500     56  0.409746                   56  0.419433
573.162500     57  0.323411                   57  0.344201
596.937500     58  0.245660                   58  0.262388
620.712500     59  0.179973                   59  0.182160
644.487500     60  0.127642                   60  0.116504
668.262500     61  0.088222                   61  0.070027
692.037500     62  0.059951                   62  0.041218
715.812500     63  0.040484                   63  0.025098
739.587500     64  0.027645                   64  0.016622
763.362500     65  0.019724                   65  0.012440
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -31.956085   0.000000 -31.956085 -31.774427 -7.041139e-07   
0.000624    2     -31.956149   0.000509 -31.955641 -31.774485  3.120700e-04   
0.000876    3     -31.956176   0.000672 -31.955504 -31.774508  4.318137e-04   
0.001229    4     -31.956210   0.000886 -31.955324 -31.774537  5.985460e-04   
0.001723    5     -31.956251   0.001161 -31.955090 -31.774573  8.271630e-04   
0.002417    6     -31.956303   0.001512 -31.954791 -31.774618  1.133930e-03   
0.003391    7     -31.956365   0.001952 -31.954413 -31.774673  1.534598e-03   
0.004757    8     -31.956441   0.002501 -31.953940 -31.774741  2.040776e-03   
0.006672    9     -31.956531   0.003175 -31.953357 -31.774821  2.658827e-03   
0.009359    10    -31.956638   0.003994 -31.952644 -31.774916  3.393627e-03   
0.013128    11    -31.956763   0.004977 -31.951786 -31.775027  4.260609e-03   
0.018415    12    -31.956904   0.006143 -31.950761 -31.775152  5.301145e-03   
0.025830    13    -31.957061   0.007512 -31.949549 -31.775290  6.578133e-03   
0.036232    14    -31.957219   0.009106 -31.948113 -31.775426  8.147212e-03   
0.050823    15    -31.957365   0.010935 -31.946429 -31.775550  1.000728e-02   
0.071291    16    -31.957507   0.013007 -31.944500 -31.775665  1.209718e-02   
0.100000    17    -31.957650   0.015385 -31.942265 -31.775772  1.439190e-02   
0.140271    18    -31.957782   0.018206 -31.939576 -31.775856  1.699835e-02   
0.196760    19    -31.957871   0.021676 -31.936194 -31.775880  2.021720e-02   
0.275997    20    -31.957852   0.026132 -31.931720 -31.775774  2.449197e-02   
0.387100    21    -31.957563   0.032108 -31.925454 -31.775367  3.029693e-02   
0.543100    22    -31.956403   0.040398 -31.916005 -31.774074  3.817783e-02   
0.761700    23    -31.953962   0.051373 -31.902589 -31.771494  4.844751e-02   
1.068500    24    -31.950595   0.065412 -31.885183 -31.767931  6.217173e-02   
1.498800    25    -31.946436   0.083927 -31.862509 -31.763513  8.175111e-02   
2.102400    26    -31.941329   0.108742 -31.832587 -31.758076  1.090338e-01   
2.949000    27    -31.935230   0.141930 -31.793300 -31.751624  1.446609e-01   
4.136600    28    -31.928698   0.186226 -31.742472 -31.744742  1.909361e-01   
5.802500    29    -31.922487   0.246194 -31.676293 -31.738280  2.494119e-01   
8.139200    30    -31.916263   0.329023 -31.587240 -31.731875  3.265686e-01   
11.417000   31    -31.909880   0.443160 -31.466721 -31.725469  4.354419e-01   
16.014700   32    -31.905028   0.599036 -31.305992 -31.720875  5.874813e-01   
22.464000   33    -31.907134   0.811673 -31.095461 -31.723605  7.930389e-01   
31.510500   34    -31.919629   1.108061 -30.811568 -31.737178  1.085427e+00   
44.200100   35    -31.938689   1.526370 -30.412319 -31.757547  1.501985e+00   
62.000000   36    -31.962525   2.107185 -29.855339 -31.782827  2.074007e+00   
85.775000   37    -31.986509   2.866808 -29.119701 -31.808154  2.813399e+00   
109.550000  38    -32.000745   3.605427 -28.395318 -31.823356  3.508504e+00   
133.325000  39    -32.010448   4.320207 -27.690242 -31.834027  4.165721e+00   
157.100000  40    -32.026163   5.235769 -26.790394 -31.851295  5.014810e+00   
180.875000  41    -32.050749   6.306369 -25.744379 -31.878164  6.036075e+00   
204.650000  42    -32.088263   7.499023 -24.589239 -31.918904  7.192527e+00   
228.425000  43    -32.141282   8.708657 -23.432625 -31.976361  8.366626e+00   
252.200000  44    -32.216420   9.911214 -22.305205 -32.057229  9.531797e+00   
275.975000  45    -32.331992  11.111263 -21.220730 -32.180621  1.071368e+01   
299.750000  46    -32.536817  12.314355 -20.222462 -32.395867  1.193259e+01   
323.525000  47    -32.843203  13.584500 -19.258703 -32.713403  1.324270e+01   
347.300000  48    -33.189404  14.993980 -18.195425 -33.071157  1.470515e+01

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')